In [7]:
import torch
import torch.nn as nn
import torchaudio


class LogMelFilterBanks(nn.Module):
    def __init__(self, n_fft=400, hop_length=160, n_mels=80, sr=16000, f_min=0.0, f_max=None, mel_scale='htk'):
        super().__init__()
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.sr = sr
        self.f_min = f_min
        self.f_max = f_max if f_max is not None else sr / 2
        self.mel_scale = mel_scale

        window = torch.hann_window(self.n_fft)
        self.register_buffer("window", window, persistent=False)

        fbanks = self._init_melscale_fbanks()
        self.register_buffer("fbanks", fbanks, persistent=False)

    def _init_melscale_fbanks(self):
        fbanks = torchaudio.functional.melscale_fbanks(
            n_freqs=self.n_fft // 2 + 1,
            n_mels=self.n_mels,
            sample_rate=self.sr,
            f_min=self.f_min,
            f_max=self.f_max,
            mel_scale=self.mel_scale
        )
        return fbanks

    def spectrogram(self, x):
        stft = torch.stft(
            x,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            window=self.window,
            return_complex=True
        )
        return stft.abs().pow(2)

    def forward(self, x):
        spec = self.spectrogram(x)
        mel = torch.matmul(spec.transpose(1, 2), self.fbanks)
        mel = mel.transpose(1, 2)
        return torch.log(mel + 1e-6)


In [9]:
!pip install ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 733.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [10]:
from tqdm import tqdm
import os
import time
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchaudio.datasets import SPEECHCOMMANDS

# from melbanks import LogMelFilterBanks


class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, root, subset=None):
        super().__init__(root, download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath, "r") as f:
                return [os.path.join(self._path, line.strip()) for line in f]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            # Исключаем файлы валидации и тестирования
            val_list = set(load_list("validation_list.txt"))
            test_list = set(load_list("testing_list.txt"))
            excludes = val_list.union(test_list)
            self._walker = [w for w in self._walker if w not in excludes]


class YesNoDataset(torch.utils.data.Dataset):
    _cache = {}

    def __init__(self, root, subset, transform=None):
        self.dataset = SubsetSC(root, subset=subset)
        self.transform = transform
        key = f"{root}_{subset}"
        if key in YesNoDataset._cache:
            self.yesno_samples = YesNoDataset._cache[key]
        else:
            self.yesno_samples = []
            for waveform, sr, label, _, _ in tqdm(self.dataset, desc="Фильтрация образцов yes/no"):
                if label in ["yes", "no"]:
                    self.yesno_samples.append((waveform, sr, label))
            YesNoDataset._cache[key] = self.yesno_samples
        self.label_map = {"no": 0, "yes": 1}

    def __len__(self):
        return len(self.yesno_samples)

    def __getitem__(self, idx):
        waveform, sr, label = self.yesno_samples[idx]
        if sr != 16000:
            waveform = torchaudio.functional.resample(waveform, sr, 16000)
        waveform = waveform.mean(dim=0, keepdim=True)
        if self.transform is not None:
            waveform = self.transform(waveform)
        target = self.label_map[label]
        return waveform, target


class SimpleCNN(nn.Module):
    def __init__(self, n_mels=80, groups=1):
        super().__init__()
        self.conv1 = nn.Conv1d(n_mels, 16, kernel_size=3, padding=1, groups=groups)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1, groups=groups)
        self.pool = nn.MaxPool1d(2)
        self.adapt_pool = nn.AdaptiveAvgPool1d(50)
        self.fc = nn.Linear(32 * 50, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.adapt_pool(x)
        x = x.flatten(start_dim=1)
        return self.fc(x)


def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def pad_crop(tensor, fixed_length):
    # tensor shape: (n_mels, frames)
    if tensor.shape[-1] > fixed_length:
        return tensor[:, :fixed_length]
    elif tensor.shape[-1] < fixed_length:
        return F.pad(tensor, (0, fixed_length - tensor.shape[-1]))
    else:
        return tensor


def train_model(n_mels=80, groups=1, epochs=5, batch_size=32, device="cuda"):
    root = "./SpeechCommands"
    if not os.path.exists(root):
        os.makedirs(root)

    train_ds = YesNoDataset(root, subset="training")
    val_ds = YesNoDataset(root, subset="validation")
    test_ds = YesNoDataset(root, subset="testing")

    log_mel_transform = LogMelFilterBanks(n_mels=n_mels).to(device)

    def collate_fn(batch):
        waveforms, targets = zip(*batch)
        targets = torch.tensor(targets, dtype=torch.long)
        max_waveform_length = max([w.shape[-1] for w in waveforms])
        padded_waveforms = [F.pad(w, (0, max_waveform_length - w.shape[-1])) for w in waveforms]
        waveforms_padded = torch.stack(padded_waveforms).squeeze(1).to(device)
        log_mels = log_mel_transform(waveforms_padded)
        fixed_length = 100
        fixed_log_mels = [pad_crop(mel, fixed_length) for mel in log_mels]
        batched_log_mels = torch.stack(fixed_log_mels)
        return batched_log_mels, targets.to(device)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    model = SimpleCNN(n_mels=n_mels, groups=groups).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs + 1):
        start = time.time()
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            out = model(x_batch)
            loss = criterion(out, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x_batch.size(0)
        total_loss /= len(train_loader.dataset)

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                preds = model(x_batch).argmax(dim=1)
                correct += (preds == y_batch).sum().item()
                total += y_batch.size(0)
        val_acc = correct / total
        epoch_time = time.time() - start
        print(f"Epoch {epoch} | TrainLoss={total_loss:.4f} | ValAcc={val_acc:.4f} | Time={epoch_time:.2f}s")

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch).argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    test_acc = correct / total
    print(f"Test Acc: {test_acc:.4f}")
    print(f"Params: {count_params(model)}")

    try:
        from ptflops import get_model_complexity_info
        macs, _ = get_model_complexity_info(model, (n_mels, 100), as_strings=True, print_per_layer_stat=False)
        print(f"FLOPs: {macs}")
    except ImportError:
        print("ptflops library not found")


if __name__ == "__main__":
    for nm in [20, 40, 80]:
        print(f"\nTraining with n_mels={nm}")
        train_model(n_mels=nm, groups=1, epochs=3)

    for g in [2, 4, 8, 16]:
        print(f"\nTraining with groups={g}")
        train_model(n_mels=80, groups=g, epochs=3)



Training with n_mels=20


Фильтрация образцов yes/no: 100%|██████████| 11005/11005 [00:36<00:00, 301.91it/s]


Epoch 1 | TrainLoss=0.1686 | ValAcc=0.9626 | Time=1.70s
Epoch 2 | TrainLoss=0.1040 | ValAcc=0.9601 | Time=1.45s
Epoch 3 | TrainLoss=0.0841 | ValAcc=0.9726 | Time=1.38s
Test Acc: 0.9684
Params: 5746
FLOPs: 187.2 KMac

Training with n_mels=40
Epoch 1 | TrainLoss=0.2105 | ValAcc=0.9639 | Time=1.54s
Epoch 2 | TrainLoss=0.1033 | ValAcc=0.9763 | Time=1.76s
Epoch 3 | TrainLoss=0.0894 | ValAcc=0.9552 | Time=1.45s
Test Acc: 0.9672
Params: 6706
FLOPs: 283.2 KMac

Training with n_mels=80
Epoch 1 | TrainLoss=0.2131 | ValAcc=0.9589 | Time=1.47s
Epoch 2 | TrainLoss=0.1217 | ValAcc=0.9751 | Time=1.46s
Epoch 3 | TrainLoss=0.0957 | ValAcc=0.9851 | Time=1.44s
Test Acc: 0.9854
Params: 8626
FLOPs: 475.2 KMac

Training with groups=2
Epoch 1 | TrainLoss=0.2028 | ValAcc=0.9614 | Time=1.43s
Epoch 2 | TrainLoss=0.1183 | ValAcc=0.9601 | Time=1.69s
Epoch 3 | TrainLoss=0.1043 | ValAcc=0.9514 | Time=1.53s
Test Acc: 0.9709
Params: 5938
FLOPs: 244.8 KMac

Training with groups=4
Epoch 1 | TrainLoss=0.2407 | ValAcc=0.